<a href="https://colab.research.google.com/github/yw7vvAW611/CS269GroupProject/blob/main/FinBERT_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack[tensorflow] > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [20]:
!pip install tqdm

In [ ]:
# Example of sentiment-classification dataset
import pandas
import numpy as np


fpb100_df = pandas.read_csv("drive/MyDrive/test.csv", sep="\t")

fpb100_array = np.array(fpb100_df)

fpb100_list = []
for idx in range(len(fpb100_array)):
  true_label = 2
  if "positive" == fpb100_array[idx, 2]:
    true_label = 0
  elif "negative" == fpb100_array[idx, 2]:
    true_label = 1
  else:
    true_label = 2
  fpb100_list.append((fpb100_array[idx, 1], true_label))

print(len(fpb100_list))

453


## 1. Improved Genetic Algorithm
 Use improved genetic algorithm augments text by replacing with synonyms from the WordNet thesaurus.

In [28]:
from textattack.augmentation.augmenter import Augmenter
from textattack.constraints.pre_transformation.stopword_modification import StopwordModification
from textattack.shared.word_embeddings import WordEmbedding
from textattack.augmentation import WordNetAugmenter
from textattack.transformations import WordSwapEmbedding
from textattack.constraints.overlap.max_words_perturbed import MaxWordsPerturbed
from textattack.constraints.semantics.word_embedding_distance import WordEmbeddingDistance
import csv
from tqdm import tqdm


#   (search_method): ImprovedGeneticAlgorithm(
#     (pop_size):  60
#     (max_iters):  20
#     (temp):  0.3
#     (give_up_if_no_improvement):  False
#     (post_crossover_check):  False
#     (max_crossover_retries):  20
#     (max_replace_times_per_index):  5
#   )
#   (goal_function):  UntargetedClassification
#   (transformation):  WordSwapEmbedding(
#     (max_candidates):  50
#     (embedding):  WordEmbedding
#   )
#   (constraints): 
#     (0): MaxWordsPerturbed(
#         (max_percent):  0.2
#         (compare_against_original):  True
#       )
#     (1): WordEmbeddingDistance(
#         (embedding):  WordEmbedding
#         (max_mse_dist):  0.5
#         (cased):  False
#         (include_unknown_words):  True
#         (compare_against_original):  False
#       )
#     (2): StopwordModification
#   (is_black_box):  True
# ) 


transformation = WordSwapEmbedding(max_candidates=50)
constraints = [MaxWordsPerturbed(max_percent=0.2, compare_against_original=True),
               WordEmbeddingDistance(max_mse_dist=0.5, cased=False, include_unknown_words=True, compare_against_original=False),
               StopwordModification()]
               
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.4, transformations_per_example=5)
# create dataframe
f = csv.writer(open("wordnet_augmented.csv", "w"))
# Write CSV Header
f.writerow(["id", "sentence", "sentiment"])
idx = 0

for s, label in tqdm(fpb100_list):
  results = augmenter.augment(s)
  f.writerow([idx, s, label])
  idx += 1
  for augmented_s in results:
    f.writerow([idx, augmented_s, label])
    idx += 1
  # print(f"Average Augment USE Score: {results[2]['avg_attack_use_score']}")
  print(f"Augmentations:")
  print(results)

  0%|          | 1/453 [00:02<18:38,  2.47s/it]

Augmentations:
['Cardona slowed her autos , turned around and comeback to the intersecting , where she called 911 ', 'Cardona slowed her car , turned around and repatriate to the intersecting , where she called 911 ', 'Cardona slowed her motors , turned nigh and repatriated to the intersection , where she called 911 ', 'Cardona slowed her vehicle , convert around and returned to the intersection , where she called 911 ', 'Cardona slowed her vehicle , revolved about and ballon to the intersection , where she called 911 ']


  0%|          | 2/453 [00:04<17:02,  2.27s/it]

Augmentations:
['Market data and analytics are derived from primary and secondary inquiries ', 'Markets data and analytics are derived from elementary and secondary looking ', 'Trade data and analytics are derived from primary and secondary investigations ', 'Traded data and analytics are derived from primary and secondhand research ', 'Traded particulars and analytics are derived from elemental and secondary research ']
Augmentations:
['Exel is headquartered in Mantyharju in Finland ', 'Exel is headquartered in Mantyharju in Finland ', 'Exel is headquartered in Mantyharju in Finn ', 'Exel is headquartered in Mantyharju in Finns ', 'Exel is headquartered in Mantyharju in Helsinki ']


  1%|          | 4/453 [00:15<23:57,  3.20s/it]

...

Augmentations:
['Nova important of the orders is over EUR 10mn ', 'Of value of the decrees is over EUR 10mn ', 'The assessed of the decree is over EUR 10mn ', 'The value of the edicts is over EUR 10mn ', 'The valuing of the order is over EUR 10mn ']


100%|█████████▉| 452/453 [51:18<00:07,  7.65s/it]

Augmentations:
["Finland printed circuit boards ( PCBs ) producers Aspocomp Group Oyj said on John 4 , 2006 it named Henry Gilchrist sheiks vice president of the cluster 's Asian operations , as of January 8 , 2007 ", "Finnish feeling circuit boards ( PCBs ) grower Aspocomp Panel Oyj pointed on December 4 , 2006 it named Hendrick Gilchrist senior vice presidente of the group 's Asian attempts , as of January 8 , 2007 ", "Finnish printed circuit boards ( PCBs ) farmer Aspocomp Groupings Oyj indicated on December 4 , 2006 it baptised Henry Gilchrist senior pornographic presidential of the grouped 's Asian operations , as of January 8 , 2007 ", "Finnish printed circuits boards ( PCBs ) maker Aspocomp Group Oyj said on Nov 4 , 2006 it named Henry Gilchrist eldest vice chairmanship of the groups 's Asiatic operations , as of June 8 , 2007 ", "Finnish prints circuit juries ( PCBs ) producer Aspocomp Group Oyj declared on September 4 , 2006 it named Henry Gilchrist upstairs vice presidents of

100%|██████████| 453/453 [51:28<00:00,  6.82s/it]

Augmentations:
['Alterations in the marketed situation and tougher price competition have spectacularly reduced demand for bread packs manufactures at the Kauhava plant , according to the company ', 'Amendment in the trade situation and tougher costing competition have substantially abated demand for bread packet manufactured at the Kauhava plant , according to the company ', 'Amendments in the market situation and tougher priced competition have substantially downsize urge for bread packaging manufactured at the Kauhava plant , according to the venture ', 'Changes in the market situation and tougher expenses competition have materially descending requests for bread packaging fabricating at the Kauhava plant , according to the company ', 'Changes in the market situation and tougher price competition have shockingly downsized demand for bakery packet manufactured at the Kauhava plant , according to the entrepreneurial ']
